In [1]:
import json
import os
import glob
import pickle

In [15]:
def laed_utterances_into_dialogs(laed_file):
    utterances = []
    # Process the individual predictions from LAED
    with open(laed_file) as f:
        dat = {}
        for line in f:
            if line.startswith("==== REPORT===="):
                break
            if len(line.strip()) == 0:
                if len(dat) > 0:
                    utterances.append(dat)
                dat = {}
                continue

            k, v = line.split(': ', 1)
            k = k.strip()
            v = v.strip()
            dat[k] = v

    laed_predictions = []
    dial = []
    for entry in utterances:
        if entry['Source'] == '<d>':
            if len(dial) > 0:
                laed_predictions.append(dial)
            dial = []

        dial.append(entry)

    # THE LAST ONE    
    laed_predictions.append(dial)
    return laed_predictions

In [14]:
def vlgan_json(path):
    vlgan_predictions = []
    with open(path) as vlganf:
        for line in vlganf:
            po = json.loads(line)
            vlgan_predictions.append(po)
    return vlgan_predictions

In [17]:
def augment(vlgan_preds, laed_outputFile, tag):
    laed_predictions = laed_utterances_into_dialogs(laed_dd_skip_file)
    
    for i in range(len(vlgan_preds)):
        laed_sample, vlgan_sample = laed_predictions[i], vlgan_preds[i]
        if len(laed_sample) < 12:
            assert (len(laed_sample)) == len(vlgan_sample)
        else:
            assert len(vlgan_sample) == 11 and len(laed_sample) > 11

        for t in range(min(12, len(vlgan_sample))):
            _t = str(t)
            vlgan_preds[i][_t][f'{tag}'] = laed_sample[t]['Predict']
            vlgan_preds[i][_t][f'{tag}.Target'] = laed_sample[t]['Target']
    return vlgan_preds

In [19]:
vlgan_predictions = vlgan_json('dd/VLGAN/dd_predictions_tmp.json')

In [23]:
data_root = 'dd/LAED/2019-12-04T23-43-57-dailydial-skip.py/'
laed_dd_skip_file = glob.glob(data_root+'/*greedy.txt.txt')[0]
print(laed_dd_skip_file)

dd/LAED/2019-12-04T23-43-57-dailydial-skip.py/2019-12-04T23-44-05-test-greedy.txt.txt


In [20]:
vlgan_predictions = augment(vlgan_predictions, laed_dd_skip_file, 'laed-skip')

In [24]:
data_root = 'dd/LAED/2019-12-05T01-18-14-dailydialog-ae.py/'
laed_dd_skip_file = glob.glob(data_root+'/*greedy.txt.txt')[0]
print(laed_dd_skip_file)

dd/LAED/2019-12-05T01-18-14-dailydialog-ae.py/2019-12-05T01-18-22-test-greedy.txt.txt


In [25]:
vlgan_predictions = augment(vlgan_predictions, laed_dd_skip_file, 'laed-ae')